# Night Shifts

In [55]:
from src.read_csv import parse_csv

import os
import pandas as pd
import copy
import statistics
from itertools import combinations

In [56]:
# Minimum number of people that need to be assigned during day shifts
min_people_day = 3

In [57]:
availabilities = [parse_csv(os.path.join('data', file)) for file in os.listdir('data') if file.endswith('.csv')]

print(f'Number of availabilities: {len(availabilities)}')
print([avail.columns[0] for avail in availabilities])

Number of availabilities: 7
['UNC', 'Michael Bryant', 'Barack Obama', 'Bill Nye', 'Euler', 'The Rock', 'Blue Devil']


In [58]:
# Sum all the availabilities element-wise, except ignoring if the value is non-numerical
# Therefore, total_avail will contain the total number of people available for each shift

total_avail = None

for df in availabilities:

    df2 = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    if total_avail is None:
        total_avail = df2
    else:
        total_avail += df2

# Add back the times (first column) to the total availability
total_avail = pd.concat([df.iloc[:, 0], total_avail], axis=1)
total_avail.columns = ['Total Available', *total_avail.columns[1:]]

# Remove the last row which is night shift
total_avail = total_avail.iloc[:-1, :]

In [59]:
# Now for the actual scheduling

display(total_avail)


,Total Available,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,7:30 - 8:00 AM,7,7,7,1,7,7,7
1,8:00 - 8:30 AM,7,7,7,7,7,7,1
2,8:30 - 9:00 AM,7,7,7,6,7,7,7
3,9:00 - 9:30 AM,7,7,7,7,7,7,6
4,9:30 - 10:00 AM,7,7,7,7,7,7,7
5,10:00 - 10:30 AM,7,7,6,7,0,7,7
6,10:30 - 11:00 AM,6,7,0,7,0,7,7
7,11:00 - 11:30 AM,7,7,0,7,0,7,7
8,11:30 - 12:00 PM,7,7,0,0,0,0,0
9,12:00 -12:30 PM,7,7,0,0,0,0,0


In [60]:
class DayPerson:
    def __init__(self, name, avail_mask):
        self.name = name
        self.avail_mask = avail_mask
        self.working_days = [[0] * 7]*38  # What days this person is working on

    def can_work(self, day, time_idx):
        return self.avail_mask[day][time_idx]
    
    def num_shifts_worked(self):
        return sum(sum(day) for day in self.working_days)

    def assign_shift(self, day, time_idx):
        self.working_days[day][time_idx] = 1

    def unassign_shift(self, day, time_idx):
        self.working_days[day][time_idx] = 0

    def __repr__(self):
        return f'{self.name}: {self.working_days}'
    

class DaySchedule:
    def __init__(self, people):
        self.people = people

    def all_worked(self, min_people_night):
        # Check if all the night shifts have been worked the minimum number of times (min_people_day)
        for day in range(7):
            for time_idx in range(38):
                if sum(person.working_days[day][time_idx] for person in self.people) < min_people_night:
                    return False
        return True

    def get_balance(self):
        # TODO inlcude number of chunks (we want to minimize this)
        shifts_worked = [person.num_shifts_worked() for person in self.people]
        return statistics.stdev(shifts_worked) if shifts_worked else 0

    def assign_shifts(self, idx, balanced_schedules, min_people_night, top=10):
        if idx >= 38*7:
            if self.all_nights_worked(min_people_night):
                balanced_schedules.append(copy.deepcopy(self))
                balanced_schedules.sort(key=lambda x: x.get_balance())

                if len(balanced_schedules) > top:
                    balanced_schedules.pop()
            return

        schedule_copy = copy.deepcopy(self)

        day = idx // 38
        time_idx = idx % 38

        # Get the people who can work on this day
        people_available = [person for person in schedule_copy.people if person.can_work(day, time_idx)]

        # Get all the combinations of people who can work on this day
        for people in combinations(people_available, min_people_night):
            for person in people:
                person.assign_shift(day, time_idx)

            schedule_copy.assign_shifts(idx+1, balanced_schedules, min_people_night, top)

            for person in people:
                person.unassign_shift(day, time_idx)


people = []

for p in availabilities:
    name = p.columns[0]
    avail_mask = p.iloc[-1:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values[0]
    people.append(DayPerson(name, avail_mask))

print(people)

# day_schedule = DaySchedule(people)
# balanced_schedules = [] # The top balanced schedules
# day_schedule.assign_shifts(0, balanced_schedules, min_people_day)


# day_map = {'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6}
# days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']



# for idx, schedule in enumerate(balanced_schedules, start=1):
#     print(f"Schedule {idx}: Balance: {schedule.get_balance()}")
#     for day in range(7):
#         print(f"{days_of_week[day]}: {', '.join(person.name for person in schedule.people if person.working_days[day])}")
#     print("\n")
    


[UNC: [0 0 0 0 0 0 0], Michael Bryant: [0 0 0 0 1 0 0], Barack Obama: [0 1 0 0 0 1 0], Bill Nye: [1 0 1 0 0 0 0], Euler: [1 0 1 1 1 1 1], The Rock: [0 1 0 1 0 0 0], Blue Devil: [1 0 0 0 0 0 0]]
